In [111]:
from dataset import getMNIST
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from lenet5.nn import ConvNeuralNet
from utils import getTorchDevice
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision.datasets import MNIST
import torchhd.functional as functional
from torch.nn.parameter import Parameter
# Note: this example requires the torchmetrics library: https://torchmetrics.readthedocs.io
import torchmetrics
from tqdm import tqdm
import struct
import torch.nn.init as init

import torchhd
from torchhd.models import Centroid
from torchhd import embeddings
# Plot Font Config 
import torchhd
import matplotlib.font_manager

font_dirs = ['./IBM_Plex_Sans/']
font_files = matplotlib.font_manager.findSystemFonts(fontpaths=font_dirs)

for font_file in font_files:
    matplotlib.font_manager.fontManager.addfont(font_file)
plt.rcParams['font.family'] = 'IBM Plex Sans'

In [112]:
train_ds, test_ds, train_ld, test_ld = getMNIST(batch_size=1)

In [113]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = getTorchDevice()
print("Using {} device".format(device))
# lenet = ConvNeuralNet(num_classes=10, use_big_hidden_layer=True).to(device)
lenet = ConvNeuralNet(num_classes=10, use_big_hidden_layer=False).to(device)

lenet.enableHDC()
lenet.load("lenet5.pth")
device, lenet

Using cpu device


(device(type='cpu'),
 ConvNeuralNet(
   (layer1): Sequential(
     (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
     (1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (2): ReLU()
     (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
   )
   (layer2): Sequential(
     (0): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
     (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (2): ReLU()
     (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
   )
   (fc): Linear(in_features=400, out_features=120, bias=True)
   (relu): ReLU()
   (fc1): Linear(in_features=120, out_features=84, bias=True)
   (relu1): ReLU()
   (fc2): Linear(in_features=84, out_features=10, bias=True)
 ))

In [166]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision.datasets import MNIST

# Note: this example requires the torchmetrics library: https://torchmetrics.readthedocs.io
import torchmetrics
from tqdm import tqdm
import torchvision.transforms as transforms
import torchhd
from torchhd.models import Centroid
from torchhd import embeddings


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using {} device".format(device))
lenet.enableHDC()
DIMENSIONS = 400
IMG_SIZE = 32
NUM_LEVELS = 1000
BATCH_SIZE = 1  # for GPUs with enough memory we can process multiple images at ones

train_ds = MNIST("../data", train=True, transform=transforms.Compose([transforms.Resize((32, 32)),
                                 transforms.ToTensor()]), download=True)
train_ld = torch.utils.data.DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)

test_ds = MNIST("../data", train=False, transform=transforms.Compose([transforms.Resize((32, 32)),
                                 transforms.ToTensor()]), download=True)
test_ld = torch.utils.data.DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)


class Encoder(nn.Module):
    def __init__(self, out_features, size, levels):
        super(Encoder, self).__init__()
        self.flatten = torch.nn.Flatten()
        self.position = embeddings.Random(size * size, out_features)
        self.value = embeddings.Level(levels, out_features)

    def forward(self, x):
        x = lenet(x)
        return torchhd.hard_quantize(x)


encode = Encoder(DIMENSIONS, IMG_SIZE, NUM_LEVELS)
encode = encode.to(device)

num_classes = len(train_ds.classes)
model = Centroid(DIMENSIONS, num_classes)
model = model.to(device)

with torch.no_grad():
    for samples, labels in tqdm(train_ld, desc="Training"):
        samples = samples.to(device)
        labels = labels.to(device)

        samples_hv = encode(samples)
        model.add(samples_hv, labels)

accuracy = torchmetrics.Accuracy("multiclass", num_classes=num_classes)

with torch.no_grad():
    model.normalize()

    for samples, labels in tqdm(test_ld, desc="Testing"):
        samples = samples.to(device)

        samples_hv = encode(samples)
        outputs = model(samples_hv, dot=True)
        accuracy.update(outputs.cpu(), labels)

print(f"Testing accuracy of {(accuracy.compute().item() * 100):.3f}%")

Using cpu device


Testing: 100%|██████████| 10000/10000 [00:08<00:00, 1219.23it/s]

Testing accuracy of 92.480%


In [167]:
def testFullLENET5():
    with torch.no_grad():
        correct = 0
        total = 0
        lenet.disableHDC()
        for images, labels in tqdm(test_ld, desc="Testing"):
            images = images.to(device)
            labels = labels.to(device)
            outputs = lenet(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print('Accuracy of the network on the 10000 test images: {} %'.format(
            100 * correct / total))
def testHDC():
    lenet.enableHDC()
    with torch.no_grad():
        model.normalize()
        for samples, labels in tqdm(test_ld, desc="Testing"):
            samples = samples.to(device)
            samples_hv = encode(samples)
            outputs = model(samples_hv, dot=True)
            accuracy.update(outputs.cpu(), labels)
    print(f"Testing accuracy of {(accuracy.compute().item() * 100):.3f}%")

In [168]:
print('Execution time (Full LENET5):', testFullLENET5(), 'seconds')

Testing: 100%|██████████| 10000/10000 [00:07<00:00, 1372.93it/s]

Accuracy of the network on the 10000 test images: 89.42 %
Execution time (Full LENET5): None seconds


In [169]:
print('Execution time (HDC):', testHDC(), 'seconds')

Testing: 100%|██████████| 10000/10000 [00:08<00:00, 1216.99it/s]

Testing accuracy of 92.480%
Execution time (Full LENET5): None seconds
